In [32]:
import pandas as pd
import os

# Step 1: Read the C019 Orginal RFQ File
Quote_Date = 20250203
RFQ_NO = "6001003909 TP TT SDS"
Base_Path = r"Z:\業務部\業務一課\G-報價\1. 外銷\C01900 Reyher\2025"
RFQ_File_Name = f"_RFQ_6001003909_20250203_014105.xlsx"
RFQ_Fie_Path = os.path.join(Base_Path,f"{Quote_Date} {RFQ_NO}",RFQ_File_Name)
DF_Original_RFQ = pd.read_excel(RFQ_Fie_Path, sheet_name="RFQ positions", skiprows=9)


# Step 2: Extract text before the second '.' for product new product series
RFQ_col_name = "Material"
if RFQ_col_name in DF_Original_RFQ.columns:
    DF_Original_RFQ["original_category"] = DF_Original_RFQ[RFQ_col_name].astype(str).apply(lambda x: ".".join(x.split(".")[:2]))

# Step 3: Get the previous product series
Category_Compare_File_Path = r"Z:\業務部\業務一課\G-報價\1. 外銷\C01900 Reyher\2025\C019 新舊產品代號對照表_Caysie.xlsx"

DIC_Item_Series = pd.read_excel(Category_Compare_File_Path, sheet_name=None, dtype=str)
DIC_Item_Series.keys
Sheet_Names = ["MS", "SDS", "TT", "DW", "CB", "TP"]
DF_Sheets = pd.concat([DIC_Item_Series[sheet_name] for sheet_name in Sheet_Names], ignore_index=True)

for num, Item_Category in DF_Original_RFQ["original_category"].items():
    for idx, Original_Category in DF_Sheets["新產品代號"].items():  
        if Item_Category == Original_Category:
            DF_Original_RFQ.loc[num, "old_category"] = DF_Sheets.loc[idx, "舊產品代號"]
            

#Step 5: Get the previous item code for DB running
if "Material" in DF_Original_RFQ.columns:
    DF_Original_RFQ["material_suffix"] = DF_Original_RFQ["Material"].astype(str).apply(lambda x: ".".join(x.split(".")[2:]))
    DF_Original_RFQ["previous_item_code"] = DF_Original_RFQ["old_category"].astype(str) + "." + DF_Original_RFQ["material_suffix"]
    
    # Reorder columns to place "previous_item_code" next to "Material"
    cols = DF_Original_RFQ.columns.tolist()
    cols.insert(cols.index("Material") + 1, cols.pop(cols.index("previous_item_code")))
    DF_Original_RFQ = DF_Original_RFQ[cols]
DF_Original_RFQ["old_category"]

C:\Users\wesley\AppData\Local\Temp\ipykernel_12292\848169429.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '07981.015' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  DF_Original_RFQ.loc[num, "old_category"] = DF_Sheets.loc[idx, "舊產品代號"]


0            NaN
1      07981.015
2      07981.015
3      07981.015
4      07981.015
         ...    
157          NaN
158          NaN
159          NaN
160          NaN
161          NaN
Name: old_category, Length: 162, dtype: object

In [33]:
import sqlite3

# Connect to DB
DB_File_Path = r"Z:\跨部門\共用資料夾\C. 業務部\詢價統計DB\QUOTATION_DATABASE.db"
with sqlite3.connect(DB_File_Path) as connect : 
    query = "SELECT PRODUCT_CODE FROM CUSTOMER_PRODUCT_SUMMARY WHERE CUSTOMER_CODE = 'C01900'"
    DF_DB = pd.read_sql_query(query, connect)

In [35]:
# Check existence of original code first than the previous code, to get searching code for SP TOOL BOX running
def check_existence(DF_Codes):
    # Ensure PRODUCT_CODE is a set for faster lookup
    product_codes_set = set(DF_DB["PRODUCT_CODE"])
    # Check if Material exists in PRODUCT_CODE
    material_match = DF_Codes["Material"].isin(product_codes_set)
    
    # Check if previous_item_code exists in PRODUCT_CODE
    final_material_match = DF_Codes["previous_item_code"].isin(product_codes_set)

    # Assign values based on conditions
    DF_Codes["SEARCHING_CODE"] = "Not found"  # Default value
    DF_Codes.loc[final_material_match, "SEARCHING_CODE"] = DF_Codes.loc[final_material_match, "previous_item_code"]
    DF_Codes.loc[material_match, "SEARCHING_CODE"] = DF_Codes.loc[material_match, "Material"]
    return DF_Codes


# # Apply function and get final DF
DF_Final = check_existence(DF_Original_RFQ)

#Save the result to a new Excel file (overwrite existing file)
output_file = os.path.join(Base_Path,f"{Quote_Date} {RFQ_NO}", f"Updated_RFQ.xlsx")

# selected_columns = ["Item", "Material", "previous_item_code", "Description", 
#                     "RFQ Quantity", "Package Qty", "SEARCHING_CODE"]

if os.path.exists(output_file):
    os.remove(output_file)

DF_Final.to_excel(output_file, index=False)

